## Gene body counts

In [ ]:
import numpy as np
import os
import sys
import collections
import matplotlib.pyplot as plt
import gzip
import loompy
# import scipy.sparse as sparse
import urllib.request
import pybedtools
from pybedtools import BedTool
import warnings
from sklearn.neighbors import NearestNeighbors
from matplotlib.collections import LineCollection

sys.path.append('/home/camiel/chromograph/')
# from chromograph.plotting.UMI_plot import UMI_plot
import chromograph
from chromograph.peak_calling.utils import *

from umap import UMAP
import sklearn.metrics
from scipy.spatial import distance
import community
import networkx as nx
from scipy import sparse
from typing import *

import logging
logger = logging.getLogger()
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%H:%M:%S')

In [ ]:
## Import path to the relevant 10X reference dataset

ref = '/data/ref/cellranger-atac/refdata-cellranger-atac-mm10-1.2.0'
indir = '/data/proj/scATAC/chromograph/mouse_test2/'
f = os.path.join(indir, '10X_test_10kb.loom')

In [ ]:
## Connect to loompy session
ds = loompy.connect(f)
logging.info(ds.shape)

In [4]:
genes = BedTool(os.path.join(ref, 'regions', 'transcripts.bed'))
print(len(genes))
genes.head()

76191
chr1	3073252	3074322	RP23-271O17.1	.	+	TEC
 chr1	3102015	3102125	Gm26206	.	+	snRNA
 chr1	3214481	3671498	Xkr4	.	-	protein_coding
 chr1	3252756	3253236	RP23-317L18.1	.	+	processed_pseudogene
 chr1	3365730	3368549	RP23-317L18.4	.	-	TEC
 chr1	3375555	3377788	RP23-317L18.3	.	-	TEC
 chr1	3464976	3467285	RP23-115I1.6	.	-	TEC
 chr1	3466586	3513553	RP23-115I1.1	.	+	antisense
 chr1	3512450	3514507	RP23-115I1.5	.	-	TEC
 chr1	3531794	3532720	RP23-115I1.2	.	+	processed_pseudogene
 

In [5]:
coding = []

for x in genes:
    if x[6] == 'protein_coding':
        coding.append(x)

coding = BedTool(coding)
coding = coding.groupby(g = [4,1], c = [2,3,4], o = ['min', 'max', 'count'], full=True)
coding.head()

chr1	3214481	3671498	Xkr4	.	-	protein_coding	3214481	3671498	1
 chr1	3999556	4409241	Rp1	.	-	protein_coding	3999556	4409241	3
 chr1	4490930	4496413	Sox17	.	-	protein_coding	4490930	4497354	4
 chr1	4773205	4785710	Mrpl15	.	-	protein_coding	4773205	4785739	3
 chr1	4807822	4846739	Lypla1	.	+	protein_coding	4807822	4846739	2
 chr1	4807891	4886770	RP24-426M1.3	.	+	protein_coding	4807891	4886770	1
 chr1	4807895	4845174	Lypla1	.	+	protein_coding	4807895	4845352	2
 chr1	4857813	4897905	Tcea1	.	+	protein_coding	4857813	4897909	2
 chr1	4909575	5019539	Rgs20	.	-	protein_coding	4909575	5070285	3
 chr1	5070017	5162340	Atp6v1h	.	+	protein_coding	5070017	5162529	3
 

In [6]:
rows = {'Gene': [], 'Chromosome': [], 'Start': [], 'End': []}

for x in coding:
    rows['Gene'].append(x[3])
    rows['Chromosome'].append(x[0])
    rows['Start'].append(int(x[1]))
    rows['End'].append(int(x[2]))
    
logging.info(f"Extracted coding genes to dict")

def Gene_count(ds, rows, res):
    i = 0
    count_dict = collections.OrderedDict()
    
    frags = ds.ca['fragments']
    frags = [strFrags_to_list(x) for x in frags]
    
    logging.info(f"extracted fragments")
    
    for cell in ds.ca['cell_id']:
        counts = {}

        for _frag in frags:
            for x in rows['Gene']:

                if _frag[0] == rows['Chromosome'][rows['Gene'] == x]:

                    st = rows['Start'][rows['Gene'] == x]
                    en = rows['End'][rows['Gene'] == x]
                    if np.logical_and(_frag[1] > st, _frag[2] < en) or np.logical_and(_frag[1] < en, _frag[2] > en) or np.logical_and(_frag[1] < st, _frag[2] > st):
                        if x not in counts.keys():
                            counts[x] = 1
                        else:
                            counts[x] += 1    
            i += 1
            res[cell] = counts
            
        if i%100 == 0:
            logging.info(f"Counted {i} cells")

In [24]:
vis = ['Neurod6', 'Gad2', 'Reln', 'Otx2', 'Olig2', 'Gfap', 'Igfbp2', 'Aqp4']

AttributeError: '<class 'loompy.attribute_manager.AttributeManager'>' object has no attribute 'cell_ID'

In [ ]:
keep = [x in vis for x in rows['Gene']]
np.array(rows['Gene'])[k]

short = {k: np.array(rows[k])[keep] for k in rows.keys()}
short

In [7]:
res = {}
counts = Gene_count(ds, short, res)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/camiel/anaconda3/envs/chromograph/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-159160c15401>", line 1, in <module>
    counts = Gene_count(ds, rows)
  File "<ipython-input-6-27c0b0876f68>", line 21, in Gene_count
    if _frag[0] == rows['Chromosome'][x]:
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/camiel/anaconda3/envs/chromograph/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/camiel/anaconda3/envs/chromograph/lib/python3.7/site-packages/IPython/core/ul

KeyboardInterrupt: 

In [ ]:
len(counts)

In [21]:
%%timeit

cell = '_cell_0'

count_dict = collections.OrderedDict()
counts = {}

frags = ds.ca['fragments'][ds.ca['cell_id']==cell]
frags = strFrags_to_list(frags[0])

i = 0
for _frag in frags:
    for x in rows['Gene']:

        if _frag[0] == rows['Chromosome'][rows['Gene'] == x]:
            
            st = rows['Start'][rows['Gene'] == x]
            en = rows['End'][rows['Gene'] == x]
            if np.logical_and(_frag[1] > st, _frag[2] < en) or np.logical_and(_frag[1] < en, _frag[2] > en) or np.logical_and(_frag[1] < st, _frag[2] > st):
                if x not in counts.keys():
                    counts[x] = 1
                else:
                    counts[x] += 1
    i += 1
    
    if i%1000 == 0:
        print(i)


.
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
.
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
.
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
.
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
.
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
.
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
.
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000

In [ ]:
# def count_fragments(frag_dict, barcodes, bsize):
#     '''
#     '''
    
#     Count_dict = collections.OrderedDict()

#     for bar in barcodes:    
#         if bar in frag_dict:
#             frags = frag_dict[bar]
#             counts = {}
#             for _frag in frags:

#                 # If a fragment spans two bins we count it twice
#                 for x in set([int(_frag[1]/bsize)*bsize+1, int(_frag[2]/bsize)*bsize+1]):
#                     k = (_frag[0], x, x + bsize - 1)
#                     if k not in counts.keys():
#                         counts[k] = 1
#                     else:
#                         counts[k] += 1
#             Count_dict[bar] = counts
#         else:
#             continue
    
#     return Count_dict;

In [9]:
feature = []
for x in coding:
    feature.append(x[3])
    
print(len(np.unique(feature)))

19883


In [ ]:
def count_fragments(frag_dict, barcodes, bsize):
    '''
    '''
    
    Count_dict = collections.OrderedDict()

    for bar in barcodes:    
        if bar in frag_dict:
            frags = frag_dict[bar]
            counts = {}
            for _frag in frags:

                # If a fragment spans two bins we count it twice
                for x in set([int(_frag[1]/bsize)*bsize+1, int(_frag[2]/bsize)*bsize+1]):
                    k = (_frag[0], x, x + bsize - 1)
                    if k not in counts.keys():
                        counts[k] = 1
                    else:
                        counts[k] += 1
            Count_dict[bar] = counts
        else:
            continue
    
    return Count_dict;
